In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from utils import MyTabNetClassifier
from acv_explainers.utils import *
from ares import AReS
from cet import CounterfactualExplanationTree
from ce import ActionExtractor
from utils import DatasetHelper, submodular_picking, DATASETS_NAME
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

seed = 2022
np.random.seed(0)

In [2]:
# Load Compas dataset

seed = 2022
np.random.seed(0)

dataset = 'c'
dataset_name = DATASETS_NAME[dataset]
model= 'X'

LAMBDA = 0.01
GAMMA = 1.0

D = DatasetHelper(dataset=dataset, feature_prefix_index=False)
X_tr, X_ts, y_tr, y_ts = D.train_test_split()

In [3]:
# Train Isolation for outlier score

from sklearn.ensemble import IsolationForest

isolation = IsolationForest()
isolation.fit(X_tr)

IsolationForest()

In [5]:
# Train the model to explain

mdl = LGBMClassifier(n_estimators=50, num_leaves=8)
mdl.fit(X_tr, y_tr)

LGBMClassifier(n_estimators=50, num_leaves=8)

In [6]:
D.feature_names

['age',
 'juv_fel_count',
 'juv_misd_count',
 'juv_other_count',
 'priors_count',
 'race:African-American',
 'race:Asian',
 'race:Caucasian',
 'race:Hispanic',
 'race:Native American',
 'race:Other',
 'c_charge_degree:F',
 'c_charge_degree:M',
 'gender']

In [7]:
D.target_name

'RecidivateWithinTwoYears'

In [8]:
D.feature_constraints

['INC',
 '',
 '',
 '',
 '',
 'FIX',
 'FIX',
 'FIX',
 'FIX',
 'FIX',
 'FIX',
 '',
 '',
 'FIX']

In [9]:
D.feature_categories

[[5, 6, 7, 8, 9, 10], [11, 12]]

In [10]:
D.feature_types

['I', 'I', 'I', 'I', 'I', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']

In [14]:
X = X_tr[mdl.predict(X_tr)==1]
X_vl = X_ts[mdl.predict(X_ts)==1]

## Actionable Recourse Summary

In [16]:
print('## Actionable Recourse Summary')
ares = AReS(mdl, X_tr, max_rule=4, max_rule_length=4, discretization_bins=10, minimum_support=0.05, print_objective=False,
            feature_names=D.feature_names, feature_types=D.feature_types, feature_categories=D.feature_categories, 
            feature_constraints=D.feature_constraints, target_name=D.target_name, target_labels=D.target_labels)
ares = ares.fit(X, max_change_num=4, cost_type='MPS', lambda_acc=1.0, lambda_cov=1.0, lambda_cst=1.0)
print('* Parameters:')
print('\t* lambda_acc: {}'.format(ares.lambda_acc)); print('\t* lambda_cov: {}'.format(ares.lambda_cov)); print('\t* lambda_cst: {}'.format(ares.lambda_cst));
print('\t* minimum support: {}'.format(ares.rule_miner_.minsup_)); print('\t* discretization bins: {}'.format(ares.rule_miner_.fd_.bins)); print('\t* pre-processing time[s]: {}'.format(ares.preprocess_time_)); 
print('\t* max rule: {}'.format(ares.max_rule_)); print('\t* max rule length: {}'.format(ares.max_rule_length_)); print('\t* Time[s]:', ares.time_); 
print('\t* uncover test: {}'.format(ares.uncover(X_vl))); print('\t* conflict: {}'.format(ares.conflict(X_vl))); print();
print('### Learned AReS')
print(ares.to_markdown())

## Actionable Recourse Summary
* Parameters:
	* lambda_acc: 1.0
	* lambda_cov: 1.0
	* lambda_cst: 1.0
	* minimum support: 0.05
	* discretization bins: 10
	* pre-processing time[s]: 0.6413756720030506
	* max rule: 4
	* max rule length: 4
	* Time[s]: 183.13370283599943
	* uncover test: 0.3354632587859425
	* conflict: 0.016826923076923076

### Learned AReS
| | Rule | Action |
| :---: | --- | --- |
| Recourse <br> rule 1 <br> (probability: 19.7%) | If 'juv_fel_count<1' <br> AND '6<=priors_count<10' | juv_fel_count<1 <br> AND priors_count<1 |
| Recourse <br> rule 2 <br> (probability: 18.2%) | If 'priors_count>=10' <br> AND 'gender=True' | priors_count<1 <br> AND gender=True |
| Recourse <br> rule 3 <br> (probability: 15.8%) | If 'age<22' <br> AND 'juv_fel_count<1' | 47<=age<53 <br> AND juv_fel_count<1 |
| Recourse <br> rule 4 <br> (probability: 12.7%) | If '22<=age<24' <br> AND 'juv_fel_count<1' | age>=53 |
| Default <br> rule | Else | priors_count<1 <br> AND gender=False |



In [25]:
x_ares = ares.predict(X_vl) + X_vl
y_ares = mdl.predict(x_ares)

print('Positive accuracy of AReS = {}'.format(np.mean(y_ares != mdl.predict(X_vl))))
print('Positive Inlier AReS = {}'.format(np.mean(isolation.predict(x_ares) == 1)))

Positive accuracy of AReS = 0.9808306709265175
Positive Inlier AReS = 0.8003194888178914


In [26]:
print('Sparsity', np.mean(np.sum(x_ares-X_vl!=0, axis=1)))

Sparsity 1.0


# Counterfactual TREE

In [21]:
print('## Counterfactual Explanation Tree')
cet = CounterfactualExplanationTree(mdl, X_tr, y_tr, max_iteration=500, lime_approximation=(model!='L'),
                                    feature_names=D.feature_names, feature_types=D.feature_types, feature_categories=D.feature_categories,
                                    feature_constraints=D.feature_constraints, target_name=D.target_name, target_labels=D.target_labels)

cet = cet.fit(X, max_change_num=8, cost_type='MPS', C=LAMBDA, gamma=GAMMA, max_leaf_size=-1, time_limit=180, verbose=True)
print('* Parameters:'); print('\t* lambda: {}'.format(cet.lambda_)); print('\t* gamma: {}'.format(cet.gamma_)); print('\t* max_iteration: {}'.format(cet.max_iteration_));
print('\t* leaf size bound:', cet.leaf_size_bound_); print('\t* leaf size:', cet.n_leaves_); print('\t* LIME approximation:', cet.lime_approximation_); print('\t* Time[s]:', cet.time_); print();
print('### Learned CET')
cet.print_tree()

## Counterfactual Explanation Tree
## Stochastic Local Searching ...
### Iteration: 10
#### Before:
- If age<35:
	- If race:Other:
		- If c_charge_degree:M:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (13/14 = 92.9% / MeanCost = 0.342):
				* age: +10
				* juv_fel_count: +2
		- Else:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (31/36 = 86.1% / MeanCost = 0.557):
				* age: +21
				* juv_misd_count: +4
				* c_charge_degree: "F" -> "M"
	- Else:
		- If juv_fel_count<8:
			- If race:Asian:
				- If age<29:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (3/4 = 75.0% / MeanCost = 0.337):
						* age: +9
						* juv_fel_count: +8
				- Else:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/1 = 0.0% / MeanCost = 0.0962):
						* age: +3
						* priors_count: -2
			- Else:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (1166/1459 = 79.9% / MeanCost = 0.724):
					* age: +62
			

- If age<35:
	- If race:Other:
		* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (42/50 = 84.0% / MeanCost = 0.526):
			* age: +19
	- Else:
		- If race:Hispanic:
			- If priors_count<2:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (27/28 = 96.4% / MeanCost = 0.0807):
					* age: +2
			- Else:
				- If c_charge_degree:M:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (9/10 = 90.0% / MeanCost = 0.295):
						* age: +10
						* priors_count: -2
				- Else:
					- If priors_count<5:
						* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (28/28 = 100.0% / MeanCost = 0.372):
							* age: +10
							* priors_count: -2
					- Else:
						* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (11/18 = 61.1% / MeanCost = 0.563):
							* age: +35
							* juv_misd_count: +9
							* priors_count: -4
							* c_charge_degree: "F" -> "M"
		- Else:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecid

- If age<35:
	- If race:Caucasian:
		- If age<29:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (225/258 = 87.2% / MeanCost = 0.586):
				* age: +20
		- Else:
			- (+) If c_charge_degree:F:
				* (+) Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (56/70 = 80.0% / MeanCost = 0.507):
					* age: +48
					* juv_misd_count: +11
					* priors_count: -3
					* c_charge_degree: "F" -> "M"
			- Else:
				* (+) Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (23/25 = 92.0% / MeanCost = 0.407):
					* age: +22
					* juv_fel_count: +6
					* juv_misd_count: +12
					* priors_count: -2
	- Else:
		- If priors_count<2:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (303/303 = 100.0% / MeanCost = 0.442):
				* age: +11
		- Else:
			- If race:Hispanic:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (45/56 = 80.4% / MeanCost = 0.328):
					* age: +10
					* priors_count: -2
			- Else:
				- If race:Other:
	

- If age<35:
	- If race:Caucasian:
		- If age<29:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (225/258 = 87.2% / MeanCost = 0.586):
				* age: +20
		- Else:
			- If c_charge_degree:F:
				- If gender:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (45/57 = 78.9% / MeanCost = 0.502):
						* age: +48
						* juv_misd_count: +11
						* priors_count: -3
						* c_charge_degree: "F" -> "M"
				- Else:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (10/13 = 76.9% / MeanCost = 0.407):
						* age: +19
						* juv_fel_count: +5
						* juv_misd_count: +12
						* priors_count: -3
						* c_charge_degree: "F" -> "M"
			- Else:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (23/25 = 92.0% / MeanCost = 0.407):
					* age: +22
					* juv_fel_count: +6
					* juv_misd_count: +12
					* priors_count: -2
	- Else:
		- If race:Hispanic:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (72/84

- If age<35:
	- If race:Caucasian:
		- If age<29:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (225/258 = 87.2% / MeanCost = 0.586):
				* age: +20
		- Else:
			- If c_charge_degree:F:
				- If gender:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (45/57 = 78.9% / MeanCost = 0.502):
						* age: +48
						* juv_misd_count: +11
						* priors_count: -3
						* c_charge_degree: "F" -> "M"
				- Else:
					- If priors_count<5:
						* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (5/5 = 100.0% / MeanCost = 0.178):
							* age: +6
							* juv_fel_count: +5
							* juv_misd_count: +6
							* priors_count: -1
					- Else:
						* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (5/8 = 62.5% / MeanCost = 0.369):
							* age: +18
							* juv_fel_count: +6
							* juv_misd_count: +12
							* priors_count: -4
			- Else:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (23/25 = 92.0% / MeanCo

...

- If juv_fel_count<8:
	- If priors_count<1:
		* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (205/209 = 98.1% / MeanCost = 0.124):
			* age: +3
	- Else:
		- If priors_count<5:
			- If priors_count<2:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (177/177 = 100.0% / MeanCost = 0.259):
					* age: +6
					* priors_count: -1
			- Else:
				- If age<35:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (527/528 = 99.8% / MeanCost = 0.451):
						* age: +15
						* priors_count: -2
				- Else:
					- If race:African-American:
						* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/1 = 0.0% / MeanCost = 0.00649):
							* juv_other_count: +2
					- Else:
						* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (1/1 = 100.0% / MeanCost = 0.0288):
							* age: +1
							* juv_fel_count: +2
		- Else:
			- If priors_count<38:
				- If race:Native American:
					- If age<29:
						* (+) Action [Recidivat

### Iteration: 450
#### Before:
- If juv_fel_count<8:
	- If priors_count<1:
		* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (205/209 = 98.1% / MeanCost = 0.124):
			* age: +3
	- Else:
		- If priors_count<5:
			- If priors_count<2:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (177/177 = 100.0% / MeanCost = 0.259):
					* age: +6
					* priors_count: -1
			- Else:
				- If age<35:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (527/528 = 99.8% / MeanCost = 0.451):
						* age: +15
						* priors_count: -2
				- Else:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (1/2 = 50.0% / MeanCost = 0.0288):
						* age: +1
						* juv_fel_count: +2
		- Else:
			- If race:Native American:
				- If age<29:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/1 = 0.0% / MeanCost = 0.209):
						* age: +5
						* juv_fel_count: +8
						* priors_count: -4
				- Else:
					- If gender:
						* A

- If juv_fel_count<8:
	- If priors_count<1:
		* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (205/209 = 98.1% / MeanCost = 0.124):
			* age: +3
	- Else:
		- If priors_count<5:
			- If priors_count<2:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (177/177 = 100.0% / MeanCost = 0.259):
					* age: +6
					* priors_count: -1
			- Else:
				- If age<35:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (527/528 = 99.8% / MeanCost = 0.451):
						* age: +15
						* priors_count: -2
				- Else:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (1/2 = 50.0% / MeanCost = 0.0288):
						* age: +1
						* juv_fel_count: +2
		- Else:
			- If race:Native American:
				- If age<45:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (1/2 = 50.0% / MeanCost = 0.252):
						* age: +8
						* juv_fel_count: +8
						* juv_other_count: +9
						* priors_count: -3
				- Else:
					- If gender:
						* Acti

### Iteration: 480
#### Before:
- If juv_fel_count<8:
	- If priors_count<1:
		* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (205/209 = 98.1% / MeanCost = 0.124):
			* age: +3
	- Else:
		- If priors_count<5:
			- If priors_count<2:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (177/177 = 100.0% / MeanCost = 0.259):
					* age: +6
					* priors_count: -1
			- Else:
				- If gender:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (451/452 = 99.8% / MeanCost = 0.452):
						* age: +15
						* priors_count: -2
				- Else:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (78/78 = 100.0% / MeanCost = 0.339):
						* age: +10
						* priors_count: -2
		- Else:
			- If race:Asian:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (1/2 = 50.0% / MeanCost = 0.104):
					* age: +3
					* juv_fel_count: +8
					* priors_count: -2
			- Else:
				- If c_charge_degree:F:
					- If race:African-Ameri

- If juv_fel_count<8:
	- If priors_count<1:
		* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (205/209 = 98.1% / MeanCost = 0.124):
			* age: +3
	- Else:
		- If priors_count<5:
			- If priors_count<2:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (177/177 = 100.0% / MeanCost = 0.259):
					* age: +6
					* priors_count: -1
			- Else:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (529/530 = 99.8% / MeanCost = 0.45):
					* age: +15
					* priors_count: -2
		- Else:
			- If age<24:
				- If race:Caucasian:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (8/8 = 100.0% / MeanCost = 0.509):
						* age: +14
						* juv_fel_count: +1
						* juv_misd_count: +5
						* priors_count: -4
				- Else:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (23/24 = 95.8% / MeanCost = 0.871):
						* age: +58
						* juv_misd_count: +5
						* priors_count: -5
			- Else:
				- If race:Hispanic:
					

In [27]:
x_cet = cet.predict(X_vl) + X_vl
y_cet = mdl.predict(x_cet)

print('Positive accuracy of CET = {}'.format(np.mean(y_cet != mdl.predict(X_vl))))
print('Positive INlier CET = {}'.format(np.mean(isolation.predict(x_cet) == 1)))

Positive accuracy of CET = 0.8594249201277955
Positive INlier CET = 0.694888178913738


In [28]:
print('sparsity', np.mean(np.sum(x_cet-X_vl!=0, axis=1)))

sparsity 1.8961661341853036


In [21]:
cet.print_tree()

- If Glucose<108:
	* Action [Diabetes: Bad -> Good] (409/409 = 100.0% / MeanCost = 0.682):
		* Pregnancies: +2
		* Glucose: +54
		* SkinThickness: +28
		* BMI: +33.8520
		* DiabetesPedigreeFunction: +0.4684
		* Age: +13
- Else:
	- If SkinThickness<35:
		- If BMI<33.7:
			* Action [Diabetes: Bad -> Good] (241/243 = 99.2% / MeanCost = 0.703):
				* Pregnancies: +3
				* Glucose: +5
				* SkinThickness: +19
				* BMI: +48.3600
				* DiabetesPedigreeFunction: +0.4076
		- Else:
			* Action [Diabetes: Bad -> Good] (56/56 = 100.0% / MeanCost = 0.23):
				* Glucose: +16
				* BloodPressure: -2
				* SkinThickness: +3
				* BMI: +35.6160
				* DiabetesPedigreeFunction: +0.0368
				* Age: +2
	- Else:
		* Action [Diabetes: Bad -> Good] (70/70 = 100.0% / MeanCost = 0.251):
			* Glucose: +11
			* BloodPressure: -3
			* SkinThickness: +42
			* BMI: +23.4240
			* DiabetesPedigreeFunction: +0.0248
			* Age: +1

